In [1]:
import json
import numpy as np
import cv2
import os
import re
from PIL import Image
from IPython.display import display

import abrupt_transition_detection as atd
import common

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [2]:
with open(common.ABRUPT_TRANSITION_TEST) as f:
    at_data = json.load(f)

arr = np.load(common.SCENE_TRANSITION_NPZ)
pix_abs = arr['pix_abs']
hist_all = arr['hist_all']
hist_reg = arr['hist_reg']

In [3]:
model_dict = {
    'abs': {
        'kmeans': KMeans(n_clusters=2, random_state=0).fit(np.expand_dims(pix_abs, axis=1)),
        'gauss': GaussianMixture(n_components=2, random_state=0).fit(np.expand_dims(pix_abs, axis=1))
    },
    'hist-all': {
        'kmeans': KMeans(n_clusters=2, random_state=0).fit(np.expand_dims(hist_all, axis=1)),
        'gauss': GaussianMixture(n_components=2, random_state=0).fit(np.expand_dims(hist_all, axis=1))
    },
    'hist-reg': {
        'kmeans': KMeans(n_clusters=2, random_state=0).fit(np.expand_dims(hist_reg, axis=1)),
        'gauss': GaussianMixture(n_components=2, random_state=0).fit(np.expand_dims(hist_reg, axis=1))
    }
}

method_keys = ['abs', 'hist-all', 'hist-reg']
model_keys = ['kmeans', 'gauss']

In [4]:
def test_jacc(method_key: str, model_key: str):
    assert method_key in method_keys
    assert model_key in model_keys
    
    model = model_dict[method_key][model_key]
    no_change_class_index = model.predict([[0]])[0]

    score = 0
    total = len(at_data)

    for data in at_data:
        diff_score = atd.gen_pixel_diff(data, score_method='hist-all')['pixel_diff_score']
        preds = model.predict(np.expand_dims(diff_score, axis=1))
        new_score = atd.non_max_suppression(diff_score, preds, no_change_class_index)
        new_pred = model.predict(np.expand_dims(new_score, axis=1))
        pred_ab_change_list = [[j + 1, j + 2] for j, pred in enumerate(new_pred) if pred != no_change_class_index]
        gt_ab_change_list = data['scene_change_pairs']
        
        score += atd.ab_change_jacc_score(pred_ab_change_list, gt_ab_change_list)

    print(F'Jacc {method_key} - {model_key}')
    print(F'Jacc score:  {score}')
    print(F'Total score: {total}')
    print(F'Norm score:  {score / total}')
    print()


def test_bin(method_key: str, model_key: str):
    assert method_key in method_keys
    assert model_key in model_keys
    
    model = model_dict[method_key][model_key]
    no_change_class_index = model.predict([[0]])[0]

    correct = 0
    total = 0

    for data in at_data:
        diff_score = atd.gen_pixel_diff(data, score_method='hist-all')['pixel_diff_score']
        preds = model.predict(np.expand_dims(diff_score, axis=1))
        new_score = atd.non_max_suppression(diff_score, preds, no_change_class_index)
        new_pred = model.predict(np.expand_dims(new_score, axis=1))
        pred_ab_change_list = [[j + 1, j + 2] for j, pred in enumerate(new_pred) if pred != no_change_class_index]
        gt_ab_change_list = data['scene_change_pairs']
        
        res = atd.ab_change_bin_acc(pred_ab_change_list, gt_ab_change_list)
        correct += res[0]
        total += res[1]

    print(F'Bin {method_key} - {model_key}')
    print(F'Correct:  {correct}')
    print(F'Total:    {total}')
    print(F'Avg Acc:  {correct / total}')
    print()

In [5]:
for method_key in method_keys:
    for model_key in model_keys:
        test_jacc(method_key, model_key)

Jacc abs - kmeans
Jacc score:  2.8538023054584847
Total score: 10
Norm score:  0.2853802305458485

Jacc abs - gauss
Jacc score:  2.8538023054584847
Total score: 10
Norm score:  0.2853802305458485

Jacc hist-all - kmeans
Jacc score:  8.114505012531328
Total score: 10
Norm score:  0.8114505012531328

Jacc hist-all - gauss
Jacc score:  6.31907980177717
Total score: 10
Norm score:  0.631907980177717

Jacc hist-reg - kmeans
Jacc score:  4.001321251847568
Total score: 10
Norm score:  0.40013212518475677

Jacc hist-reg - gauss
Jacc score:  0.04856237816764132
Total score: 10
Norm score:  0.004856237816764132



In [6]:
for method_key in method_keys:
    for model_key in model_keys:
        test_bin(method_key, model_key)

Bin abs - kmeans
Correct:  164
Total:    330
Avg Acc:  0.49696969696969695

Bin abs - gauss
Correct:  164
Total:    330
Avg Acc:  0.49696969696969695

Bin hist-all - kmeans
Correct:  299
Total:    330
Avg Acc:  0.906060606060606

Bin hist-all - gauss
Correct:  238
Total:    330
Avg Acc:  0.7212121212121212

Bin hist-reg - kmeans
Correct:  231
Total:    330
Avg Acc:  0.7

Bin hist-reg - gauss
Correct:  3
Total:    330
Avg Acc:  0.00909090909090909



In [7]:
# plt.figure(figsize=(18, 10))
# plt.hist(all_diff_score, bins=30)

# data = at_data[0]
# diff_list = diff_score_list[0]
# real_change = data['scene_change_pairs']

# all_pairs = [[i, i + 1] for i in range(1, 174)]
# for i, pair in enumerate(all_pairs):
#     if pair in real_change:
#         plt.axvline(diff_list[i], linewidth=1, color='k', linestyle='dashed')